<a href="https://colab.research.google.com/github/fbelinchon/HuggingFace_ES/blob/main/Simple_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install transformers[sentencepiece]

     |████████████████████████████████| 3.8 MB 25.7 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 6.5 MB 59.9 MB/s 
     |████████████████████████████████| 895 kB 63.3 MB/s 
     |████████████████████████████████| 596 kB 52.7 MB/s 
     |████████████████████████████████| 1.2 MB 51.1 MB/s 


# Vamos a implementar un modelo de traducción usando diferentes métodos


## Método Simple: Pipeline

Es la forma más sencilla pero que nos ofrece menos personalizaciones.

Pasos:
* Importamos el objeto pipeline
* Le pasamos el tipo de tarea ("translation") y elmodelo a utilizar (Helsinki-NLP/opus-mt-es-en)

Al objeto resultante le podemos pasar una lista de sentencias en español y nos devolvera el listado de traducciones.

In [11]:
from transformers import pipeline

In [19]:
checkpoint = "Helsinki-NLP/opus-mt-es-en"
translator = pipeline("translation",model=checkpoint)

In [20]:
print(translator(["Hola este es mi perro","No encuentro donde he dejado mi coche"]))

[{'translation_text': 'Hey, this is my dog.'}, {'translation_text': "I can't find where I left my car."}]


## Método más detallado

En este caso vamos a realizar manualmente los pasos que se realizan con el objeto pipeline.

Pasos:
* Obtenemos el tokenizador asociado al modelo que vamos a utilizar.
* Obtenemos el modelo en sí (ya preentrenado).
* Convertimos nuestro texto a token.
* Los tokens del paso anterior los convertimos a números.
* El listadode números serála entrada al modelo.
* La salida 

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en",use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

In [ ]:

batch = tokenizer("Hola este es mi perro", return_tensors="pt")
gen = model.generate(**batch)
tokenizer.batch_decode(gen, skip_special_tokens=True)

['Hey, this is my dog.']